# Main

This model will take in a valence-arousal vector and a Bouba-Kiki vector. It will then output an encoded earcon representation, which will be passed to the MusicGen Decoder to generate the final earcon.

The output of the MusicGen Decoder will then be encoded by EncodecFeatureExtractor, and the vectors will be used to calculate the loss

In [ ]:
# import relevant libraries
import pandas as pd
import numpy as np
import ast
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pd.set_option('display.max_columns', None)

## Dataset

Each row in the dataset will consist of:
- An Earcon represented by an Encodec vector
- An image represented in a Valence Arousal Vector
- A Bouba-Kiki Value derived from the image
- A Pseudoword

The rows will be paired by cosine similarity between the Earcon's Encodec vector and the VA Vector from the image. The Bouba-Kiki Value and Pseudoword will be generated after the images are paired with the audio

### Earcons

In [ ]:
# load in earcons
earcons = pd.read_csv('dataset\earcon_dataset\earcon_dataset.csv')

earcons['query'] = earcons['query'].apply(ast.literal_eval)
earcons["query"] = earcons["query"].apply(lambda x: x[0])

earcons = earcons[["query", "name"]]

earcons["filepaths"] = earcons["name"].apply(lambda x: f"dataset/earcon_dataset/earcons/{x}")

earcons

In [ ]:
# prep encodec model
from encodec import EncodecModel


encodec_model = EncodecModel.encodec_model_24khz().to(device)

In [ ]:
import torchaudio


def extract_earcon_features(filepaths, encodec_model, target_sample_rate=24000, target_length=512):
    earcon_features = []

    for path in filepaths:
        # load in the audio file
        waveform, sample_rate = torchaudio.load(path)
        waveform = waveform.to(device)

        # if stereo, convert to mono
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True).to(device)

        # resample if necessary
        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, target_sample_rate).to(device)
            waveform = resampler(waveform).to(device)

        # add batch dimension so that the shape is
        # [1, 1, num_samples] because encodec
        # expects that format
        waveform = waveform.unsqueeze(0)

        # encode the waveform
        with torch.no_grad():
            encoded_frames = encodec_model.encode(waveform)
            compressed_features = encoded_frames[0][0].to(device)  # Take the first codebook
        
        # truncate and pad
        length = compressed_features.shape[2]
        if length > target_length:
            compressed_features = compressed_features[:, :, :target_length].to(device)
        else:
            pad = torch.zeros((compressed_features.shape[0], compressed_features.shape[1], target_length - length)).to(device)
            compressed_features = torch.cat((compressed_features, pad), dim=2).to(device)

        # remove the first dimension
        compressed_features = compressed_features.squeeze(0)
        earcon_features.append(compressed_features.to("cpu"))

    return earcon_features


# Apply the function to all rows in the earcons dataframe
earcons["earcon_features"] = extract_earcon_features(earcons["filepaths"], encodec_model)
earcons

In [ ]:
earcons["earcon_features"][0].shape

In [ ]:
# find the largest value embedded in the earcon features

smallest = 1000000
largest = 0
for i in range(len(earcons["earcon_features"])):
    if largest < int(earcons["earcon_features"][i].max()):
        largest = int(earcons["earcon_features"][i].max())
    if smallest > int(earcons["earcon_features"][i].min()):
        smallest = int(earcons["earcon_features"][i].min())

print(largest)
print(smallest)

### Images

In [ ]:
# load in images
images = pd.read_csv('dataset\landscape1\csvs\image_classification.csv')

# extract top tag and similarity score
images['top_tags'] = images['top_tags'].apply(ast.literal_eval)
images["top_tags"] = images["top_tags"].apply(lambda x: x[0])
images["similarity_scores"] = images["similarity_scores"].apply(ast.literal_eval)
images["similarity_scores"] = images["similarity_scores"].apply(lambda x: x[0])

images["image_path"] = images["image_path"].str.lstrip("../")

In [ ]:

# load CLIP model
from transformers import CLIPProcessor, CLIPModel
from PIL import Image


# Function to calculate image vectors
def calculate_image_vectors(image_paths, clip_model, clip_processor):
    image_features = []
    count = 0
    for image_path in image_paths:
        count += 1
        image = clip_processor(images=Image.open(image_path), return_tensors="pt").pixel_values.to(device)
        with torch.no_grad():
            image = clip_model.get_image_features(image)
            image = image.squeeze(0)
            image_features.append(image.to("cpu"))
        if count % 1000 == 0:
            print(f"Processed {count} images")
    return image_features


# Apply the function to the images dataframe
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

images["image_features"] = calculate_image_vectors(images["image_path"].tolist(), clip_model, clip_processor)
images

In [ ]:
images["image_features"][0].shape

### Calculate Cosine Similarity

This will be used to build the dataset for our model

In [ ]:
# calculate the cosine similarity & store in a new df
from sklearn.metrics.pairwise import cosine_similarity


def compute_torch_cosine_similarity(image_vectors, earcon_vectors):
    # Convert to PyTorch tensors
    image_tensor = torch.tensor(np.stack(image_vectors))
    earcon_tensor = torch.tensor(np.mean(np.stack(earcon_vectors), axis=1)).float()
    
    # Normalize vectors
    image_tensor_norm = image_tensor / image_tensor.norm(dim=1, keepdim=True)
    earcon_tensor_norm = earcon_tensor / earcon_tensor.norm(dim=1, keepdim=True)
    
    # Compute cosine similarity
    similarity_matrix = torch.mm(image_tensor_norm, earcon_tensor_norm.t())
    
    return similarity_matrix.numpy()


def process_similarities(images, earcons):
    # Compute similarity matrix (using one of the methods above)
    similarity_matrix = compute_torch_cosine_similarity(
        images['image_features'].tolist(), 
        earcons['earcon_features'].tolist()
    )
    
    # Find the index of the most similar earcon for each image
    most_similar_indices = np.argmax(similarity_matrix, axis=1)
    
    # Extract the most similar earcon details
    result_df = pd.DataFrame({
        'split': images['split'],
        'earcon_filename': earcons.iloc[most_similar_indices]['name'].values,
        'earcon_filepath': earcons.iloc[most_similar_indices]['filepaths'].values,
        'earcon_features': earcons.iloc[most_similar_indices]['earcon_features'].values,
        'image_filename': images['filename'],
        'image_filepath': images['image_path'],
        'image_features': images['image_features'],
        'image_tag': images['top_tags'],
        'image_tag_similarity': images['similarity_scores'],
        'similarity_score': similarity_matrix[np.arange(len(most_similar_indices)), most_similar_indices]
    })
    
    return result_df

earcon_image_dataset = process_similarities(images, earcons)
earcon_image_dataset

In [ ]:
earcon_image_dataset.earcon_filename.value_counts()

### Build Pseudowords and Bouba-Kiki value

In [ ]:
import random
from joblib import Parallel, delayed
import utils.psword_gen as psword_gen
import utils.psword_utils as psword_utils


def generate_pseudoword_and_bouba_kiki(image_path, sound_dict):
    x_values, y_values = psword_utils.process_image(image_path, 50, 150)
    weighted_angles, roundness = psword_utils.calculate_weighted_angles_by_edge_length(x_values, y_values)
    
    random.seed(42)
    
    psword = psword_gen.pseudoword_generator(
        roundness,
        len(x_values),
        sound_dict=sound_dict
    )
    
    return roundness, psword, weighted_angles

# Parallelized function
def process_row(row):
    return generate_pseudoword_and_bouba_kiki(row['image_filepath'], sound_dict)


# sound dict
sound_dict = psword_gen.load_sound_mappings('utils/sound_mappings.json')

# Parallelize using joblib
results = Parallel(n_jobs=-1)(delayed(process_row)(row) for _, row in earcon_image_dataset.iterrows())

# Extract and assign results
earcon_image_dataset[['roundness', 'pseudoword', 'weighted_angles']] = pd.DataFrame(results, index=earcon_image_dataset.index)

earcon_image_dataset

### Save to pickle

In [ ]:
earcon_image_dataset.to_pickle('dataset/combined_dataset/earcon_image_dataset2.pkl')

## Training

### Load from pickle

In [ ]:
import pandas as pd

earcon_image_dataset = pd.read_pickle('dataset/combined_dataset/earcon_image_dataset2.pkl')
earcon_image_dataset

### Build Dataloaders

In [ ]:
# Split the dataset based on the "split" column
train_df = earcon_image_dataset[earcon_image_dataset['split'] == 'train']
train_df = train_df.drop(columns='split')
val_df = earcon_image_dataset[earcon_image_dataset['split'] == 'validation']
val_df = val_df.drop(columns='split')
test_df = earcon_image_dataset[earcon_image_dataset['split'] == 'test']
test_df = test_df.drop(columns='split')

In [ ]:
train_df = train_df.sample(frac=0.01, random_state=42)
val_df = val_df.sample(frac=0.05, random_state=42)
test_df = test_df.sample(frac=1, random_state=42)
len(train_df), len(val_df), len(test_df)

In [ ]:
# Create dataloaders
from utils.musicgen_utils import create_earcon_dataloaders
train_loader, val_loader, test_loader = create_earcon_dataloaders(train_df, val_df, test_df, batch_size=1, num_workers=8)

### Model

The model pipeline is as follows:
- The Earcon Encodec Vector is the target
- The VA Vector and Bouba-Kiki Value will be inputs to the model
- The model will output a set of vectors which will be fed to the MusicGen Decoder along with the Pseudoword
- The output of MusicGen Decoder will be encoded by Encodec
- The output of Encodec will be considered the final output, and loss will be calculated based on the difference between this output and the target Encodec vector from the Earcon

In [1]:
# init model
from utils.musicgen_model import *
from utils.musicgen_utils import *


model = MultimodalEarconGenerator(
    freeze_musicgen=False
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\qkm20\anaconda3\envs\fyp\Lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\Users\qkm20\anaconda3\envs\fyp\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(
c:\Users\qkm20\anaco

### Training the model

In [ ]:
# optimizer
import torch.optim as optim


trainable_params = [
    param for param in model.parameters() if param.requires_grad
]
optimizer = optim.Adam(
    trainable_params,
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-08,
    weight_decay=0
)

In [3]:
print(model)

MultimodalEarconGenerator(
  (encodec): EncodecModel(
    (encoder): SEANetEncoder(
      (model): Sequential(
        (0): SConv1d(
          (conv): NormConv1d(
            (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
            (norm): Identity()
          )
        )
        (1): SEANetResnetBlock(
          (block): Sequential(
            (0): ELU(alpha=1.0)
            (1): SConv1d(
              (conv): NormConv1d(
                (conv): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
                (norm): Identity()
              )
            )
            (2): ELU(alpha=1.0)
            (3): SConv1d(
              (conv): NormConv1d(
                (conv): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
                (norm): Identity()
              )
            )
          )
          (shortcut): SConv1d(
            (conv): NormConv1d(
              (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
              (norm): Identity()
            )
          )


In [ ]:
# train model
train_multimodal_earcon_model(
    model=model,
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    optimizer=optimizer,
    epochs=10,
)

In [ ]:
save_multimodal_model(model, model.loss_fn, prefix="MusicGenModel_04")

## Testing

In [ ]:
# Loading model

model = MultimodalEarconGenerator()
model = load_multimodal_model(model, model_path="outputs\MusicGenModel_04_model.pt")

In [ ]:
import random

# choose 10 unique random image/earcon pairs for testing
elements = random.sample(range(0, len(test_df)), 10)
elements

In [ ]:
# generate the audio for the selected pairs

audio_list = []

for i in range(len(elements)):
    audio_list.append(
        generate_earcon(
            model,
            test_df["image_features"].iloc[elements[i]],
            test_df["roundness"].iloc[elements[i]],
            image_tag=test_df["image_tag"].iloc[elements[i]],
        )
    )

In [ ]:
from IPython.display import Audio, display, Image


def display_image(image_path):
    display(Image(filename=image_path))


def play_audio(filepath):
    display(Audio(filepath))


def play_generated_audio(audio, sampling_rate=24000):
    display(Audio(audio[0].cpu().numpy(), rate=sampling_rate))

In [ ]:
# display the images and play the audio for every pair
for i in range(len(elements)):
    # display image
    print(f"Image {i+1}:")
    display_image(test_df["image_filepath"].iloc[elements[i]])
    # play paired earcon
    print(f"Original Paired Earcon {i+1}:")
    play_audio(test_df["earcon_filepath"].iloc[elements[i]])
    # play generated earcon
    print(f"Generated Earcon {i+1}:")
    play_generated_audio(audio_list[i])